In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
def dividir_rango_fechas(desde, hasta):
    """Divide un rango de fechas en segmentos de hasta un año."""
    inicio = []
    final = []
    fecha_actual = desde
    while fecha_actual < hasta:
        siguiente = min(fecha_actual + timedelta(days=365), hasta)
        inicio.append(fecha_actual)
        final.append(siguiente)
        fecha_actual = siguiente + timedelta(days=1)
    return inicio, final

def safe_divide(numerator, denominator, default_value=float(0)):
    try:
        return numerator / denominator
    except ZeroDivisionError:
        return default_value

def average_interest(serie):
    # Convertir la serie a un array de valores no nulos
    valores = serie.dropna().values
    # Calcular el interés compuesto
    valores = (valores/100)+1
    return ((pow(np.prod(valores), safe_divide(1,len(serie),0)) - 1)*10000)

def log_return_std(serie):
    """
    Calcula la desviación estándar de rendimientos logarítmicos.
    """
    # valores = serie.dropna().values
    log_returns = np.log(1 + serie)  # Rendimientos logarítmicos
    return np.std(log_returns)


In [3]:
principales_variables = pd.read_csv('principalesVariables.csv')
idvariables = principales_variables['idVariable'].tolist()
variablesIndicadores = [1,4,5,15,16,17,18,19,21,22,23,24,25,26,30,31,32,40,42,43]
variablesTasas = [6,7,8,9,10,11,12,13,14,34,35,41]
variablesIPC = [27,28,29]
fecha_inicio = datetime(1996,1,1)
fecha_final = datetime.now() - timedelta(days=datetime.now().weekday() + 2 if datetime.now().weekday() < 5 else 1)
inicio, final = dividir_rango_fechas(fecha_inicio, fecha_final)
empty_df = pd.DataFrame()
df_dicc={}
fechas = pd.date_range(start=fecha_inicio, end=fecha_final, freq='D')
registros_bcra = pd.DataFrame({'fecha': fechas})

In [88]:
registros_bcra = pd.DataFrame({'fecha': fechas})

In [5]:
for v in idvariables:
    for i, f in zip(inicio, final):
        url = f"https://api.bcra.gob.ar/estadisticas/v2.0/datosvariable/{v}/{i.strftime('%Y-%m-%d')}/{f.strftime('%Y-%m-%d')}"
        try:
            response = requests.get(url, verify=False)

            # Convertir la respuesta JSON a un objeto de Python
            data = response.json()
        
            results = data.get("results")
            if results:
                print(f'Cargando datos {i} a {f}...')
                # Crear un DataFrame temporal con los resultados
                temp_df = pd.DataFrame(results)
                # Concatenar el DataFrame temporal con el acumulador
                empty_df = pd.concat([empty_df, temp_df])
            else:
                print(f'No hay datos en rango {i} a {f}')

        except KeyError as e:
            print(f"Error procesando los datos para el rango {i} - {f}: {e}")

    df_dicc[v] = empty_df
    empty_df = empty_df.iloc[0:0]
    print('='*100)
    print(f'Variable {v} cargada.')
    print('='*100)
    print('='*100)

    

Cargando datos 1996-01-01 00:00:00 a 1996-12-31 00:00:00...
Cargando datos 1997-01-01 00:00:00 a 1998-01-01 00:00:00...
Cargando datos 1998-01-02 00:00:00 a 1999-01-02 00:00:00...
Cargando datos 1999-01-03 00:00:00 a 2000-01-03 00:00:00...
Cargando datos 2000-01-04 00:00:00 a 2001-01-03 00:00:00...
Cargando datos 2001-01-04 00:00:00 a 2002-01-04 00:00:00...
Cargando datos 2002-01-05 00:00:00 a 2003-01-05 00:00:00...
Cargando datos 2003-01-06 00:00:00 a 2004-01-06 00:00:00...
Cargando datos 2004-01-07 00:00:00 a 2005-01-06 00:00:00...
Cargando datos 2005-01-07 00:00:00 a 2006-01-07 00:00:00...
Cargando datos 2006-01-08 00:00:00 a 2007-01-08 00:00:00...
Cargando datos 2007-01-09 00:00:00 a 2008-01-09 00:00:00...
Cargando datos 2008-01-10 00:00:00 a 2009-01-09 00:00:00...
Cargando datos 2009-01-10 00:00:00 a 2010-01-10 00:00:00...
Cargando datos 2010-01-11 00:00:00 a 2011-01-11 00:00:00...
Cargando datos 2011-01-12 00:00:00 a 2012-01-12 00:00:00...
Cargando datos 2012-01-13 00:00:00 a 201

In [89]:
for clave, df in df_dicc.items():
    df['fecha'] = pd.to_datetime(df['fecha'])
    # Combinar los datos basados en la columna 'fecha'
    registros_bcra = registros_bcra.merge(
        df[['fecha', 'valor']],
        on='fecha',
        how='left',
        suffixes=('', f'_{clave}')  # Evita conflictos en los nombres
    )
    # Renombrar la columna 'valor' resultante al nombre de la clave
    registros_bcra.rename(columns={'valor': clave}, inplace=True)

# Mostrar el resultado
print(registros_bcra)

           fecha        1        4        5     6        7        8     9  10  \
0     1996-01-01      NaN      NaN      NaN   NaN      NaN      NaN   NaN NaN   
1     1996-01-02  17314.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
2     1996-01-03  17404.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
3     1996-01-04  17435.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
4     1996-01-05  17609.0      NaN      NaN   NaN      NaN      NaN   NaN NaN   
...          ...      ...      ...      ...   ...      ...      ...   ...  ..   
10564 2024-12-03  31240.0  1042.35  1013.17  35.0  36.9375  37.8750  40.0 NaN   
10565 2024-12-04  31266.0  1042.18  1013.25  35.0  34.6875  36.0625  40.0 NaN   
10566 2024-12-05      NaN  1043.32  1013.83  35.0  35.3750  36.7500  40.0 NaN   
10567 2024-12-06      NaN  1044.61  1015.75  32.0      NaN      NaN  36.0 NaN   
10568 2024-12-07      NaN      NaN      NaN   NaN      NaN      NaN   NaN NaN   

          11  ...  29      

In [90]:
# Rellenar valores nulos con interpolacion lineal
for col in idvariables:
    if col in registros_bcra.columns:
        # Asegurarnos de que hay al menos un valor no nulo antes y después
        if registros_bcra[col].notna().sum() > 1:
            # Rellenar valores nulos con interpolación lineal
            registros_bcra[col] = registros_bcra[col].interpolate(method='linear', limit_direction='forward')

print(registros_bcra)


           fecha        1        4        5     6        7        8     9  \
0     1996-01-01      NaN      NaN      NaN   NaN      NaN      NaN   NaN   
1     1996-01-02  17314.0      NaN      NaN   NaN      NaN      NaN   NaN   
2     1996-01-03  17404.0      NaN      NaN   NaN      NaN      NaN   NaN   
3     1996-01-04  17435.0      NaN      NaN   NaN      NaN      NaN   NaN   
4     1996-01-05  17609.0      NaN      NaN   NaN      NaN      NaN   NaN   
...          ...      ...      ...      ...   ...      ...      ...   ...   
10564 2024-12-03  31240.0  1042.35  1013.17  35.0  36.9375  37.8750  40.0   
10565 2024-12-04  31266.0  1042.18  1013.25  35.0  34.6875  36.0625  40.0   
10566 2024-12-05  31266.0  1043.32  1013.83  35.0  35.3750  36.7500  40.0   
10567 2024-12-06  31266.0  1044.61  1015.75  32.0  35.3750  36.7500  36.0   
10568 2024-12-07  31266.0  1044.61  1015.75  32.0  35.3750  36.7500  36.0   

         10     11  ...    29        30       31      32     34     35     

In [ ]:
resultado = pd.DataFrame()

resultado[variablesIPC] = registros_bcra.groupby(pd.Grouper(key='fecha', freq='ME'))[variablesIPC].agg(average_interest)
print(resultado)

In [91]:
dicc_registros_bcra={}

In [92]:
registros_bcra_mensual = pd.DataFrame()
registros_bcra_trimestral = pd.DataFrame()
registros_bcra_anual = pd.DataFrame()
porcentajes_mensual = pd.DataFrame()
porcentajes_trimestral = pd.DataFrame()
porcentajes_anual = pd.DataFrame()
ipc_mensual = pd.DataFrame()
ipc_trimestral = pd.DataFrame()
ipc_anual = pd.DataFrame()

registros_bcra_mensual[variablesIndicadores] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='ME', fill_method = None))[variablesIndicadores].agg('last'))
registros_bcra_trimestral[variablesIndicadores] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='QE', fill_method = None))[variablesIndicadores].agg('last'))
registros_bcra_anual[variablesIndicadores] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='YE', fill_method = None))[variablesIndicadores].agg('last'))


ipc_mensual[variablesIPC] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='ME'))[variablesIPC].agg('mean'))
ipc_trimestral[variablesIPC] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='QE'))[variablesIPC].agg('mean'))
ipc_anual[variablesIPC] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='YE'))[variablesIPC].agg('mean'))

porcentajes_mensual[variablesTasas] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='ME'))[variablesTasas].agg('mean'))
porcentajes_trimestral[variablesTasas] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='QE'))[variablesTasas].agg('mean'))
porcentajes_anual[variablesTasas] = (registros_bcra.groupby(pd.Grouper(key='fecha', freq='YE'))[variablesTasas].agg('mean'))


registros_bcra_mensual = pd.concat([registros_bcra_mensual, porcentajes_mensual, ipc_mensual], axis=1)
registros_bcra_trimestral = pd.concat([registros_bcra_trimestral, porcentajes_trimestral, ipc_trimestral], axis=1)
registros_bcra_anual = pd.concat([registros_bcra_anual, porcentajes_anual, ipc_anual], axis=1)


dicc_registros_bcra = {
    'diario': registros_bcra,
    'mensual': registros_bcra_mensual.reset_index(),
    'trimestral': registros_bcra_trimestral.reset_index(),
    'anual': registros_bcra_anual.reset_index()
}

In [94]:
dicc_registros_bcra['diario'][['fecha']+[27]].head(62)

,fecha,27
0,1996-01-01,NaN
1,1996-01-02,NaN
2,1996-01-03,NaN
3,1996-01-04,NaN
4,1996-01-05,NaN
...,...,...
57,1996-02-27,-0.258621
58,1996-02-28,-0.279310
59,1996-02-29,-0.300000
60,1996-03-01,-0.306452


In [95]:
# Calcular tabla de metricas moviles segun 'xdays'


xdays = 5  # Número de días para la ventana móvil

# Crear DataFrames para almacenar las métricas
dicc_metrics = {}
for indice, df in dicc_registros_bcra.items():
    # Asegúrate de que 'fecha' esté en formato datetime
    df['fecha'] = pd.to_datetime(df['fecha'])
    dicc_metrics[f'metricas_{indice}'] = pd.DataFrame({'fecha': df['fecha']})

for indice, df in dicc_registros_bcra.items():
    for columna in idvariables:
        if columna in df.columns:
            df = df.sort_values('fecha')  # Asegurar el orden cronológico
            df[columna] = df[columna].astype(float)

            # Promedio móvil
            dicc_metrics[f'metricas_{indice}'][f"{columna}_{xdays}MA"] = (df.rolling(xdays)[columna].mean())

            # Desviación estándar
            desviacion_estandar = df.rolling(xdays)[columna].std()

            # Calcular la desviación estándar relativa como porcentaje de la media
            desviacion_estandar_relativa = (desviacion_estandar / df.rolling(xdays)[columna].mean())
        
            dicc_metrics[f'metricas_{indice}'][f"{columna}_desviacion_estandar_relativa"] = desviacion_estandar_relativa

             # Promedio de la desviación estándar relativa (promedio móvil de la desviación estándar relativa)
            dicc_metrics[f'metricas_{indice}'][f"{columna}_{xdays}MA_desviacion_estandar_relativa"] = (desviacion_estandar_relativa.rolling(xdays).mean())

            # Calcular Rate of Change (ROC) en diferentes periodos

            # Diario
            dicc_metrics[f'metricas_{indice}'][f"{columna}_RoC_{indice}"] = df[columna].pct_change()


print(dicc_metrics)



C:\Users\1sant\AppData\Local\Temp\ipykernel_6904\942088181.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dicc_metrics[f'metricas_{indice}'][f"{columna}_RoC_{indice}"] = df[columna].pct_change()
C:\Users\1sant\AppData\Local\Temp\ipykernel_6904\942088181.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dicc_metrics[f'metricas_{indice}'][f"{columna}_{xdays}MA"] = (df.rolling(xdays)[columna].mean())
C:\Users\1sant\AppData\Local\Temp\ipykernel_6904\942088181.py:28: PerformanceWarning: DataFrame is highly fragmented.  This 

{'metricas_diario':            fecha         1_5MA  1_desviacion_estandar_relativa  \
0     1996-01-01           NaN                             NaN   
1     1996-01-02           NaN                             NaN   
2     1996-01-03           NaN                             NaN   
3     1996-01-04           NaN                             NaN   
4     1996-01-05           NaN                             NaN   
...          ...           ...                             ...   
10564 2024-12-03  30859.200000                        0.014969   
10565 2024-12-04  31069.600000                        0.009938   
10566 2024-12-05  31206.666667                        0.004724   
10567 2024-12-06  31270.400000                        0.000859   
10568 2024-12-07  31260.800000                        0.000372   

       1_5MA_desviacion_estandar_relativa  1_RoC_diario        4_5MA  \
0                                     NaN           NaN          NaN   
1                                     NaN  

In [96]:
for indice, df in dicc_registros_bcra.items():
    df.to_csv(f'({indice}) Registros BCRA desde 1996.csv', index=False)

for indice, df in dicc_metrics.items():
    df.to_csv(f'{indice} BCRA desde 1996.csv', index=False)